In [30]:
#process data

import pandas as pd

x_train = pd.read_parquet("../data/train_data/phase-1/prob-2/train_x.parquet")
y_train = pd.read_parquet("../data/train_data/phase-1/prob-2/train_y.parquet")

x_test = pd.read_parquet("../data/train_data/phase-1/prob-2/test_x.parquet")
y_test = pd.read_parquet("../data/train_data/phase-1/prob-2/test_y.parquet")

n_label_1 = len(y_train.loc[y_train["label"] == 1])
n_label_0 = len(y_train) - n_label_1
scale_pos_weight = n_label_0 / n_label_1
print("scale pos weight:", scale_pos_weight)


scale pos weight: 2.2994226010448173


In [35]:
# register model to mlflow
from mlflow.models.signature import infer_signature
import mlflow
# init mlflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment(
    f"phase-1_prob-2_model-1"
)

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1687686717553, experiment_id='2', last_update_time=1687686717553, lifecycle_stage='active', name='phase-1_prob-2_model-1', tags={}>

In [32]:
# train model
import xgboost as xgb
model = xgb.XGBClassifier(objective="binary:logistic", scale_pos_weight=scale_pos_weight)

# from sklearn.linear_model import LogisticRegression
# lr_model = LogisticRegression()

# from sklearn.ensemble import RandomForestClassifier
# rf_model = RandomForestClassifier()

model.fit(x_train, y_train)
# lr_model.fit(x_train, y_train)
# rf_model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [33]:
# evaluate model
from sklearn.metrics import classification_report, roc_auc_score
predictions = model.predict(x_test)
print(classification_report(y_test, predictions))
auc_score = roc_auc_score(y_test, predictions)
metrics = {"test_auc": auc_score}
print(f"metrics: {metrics}")

              precision    recall  f1-score   support

           0       0.89      0.80      0.84      8333
           1       0.63      0.77      0.69      3667

    accuracy                           0.79     12000
   macro avg       0.76      0.78      0.76     12000
weighted avg       0.81      0.79      0.79     12000

metrics: {'test_auc': 0.7832101012428826}


In [8]:
# evaluate model
from sklearn.metrics import classification_report, roc_auc_score
n_models = 3
predictions = [None] * n_models
predictions[0] = xgb_model.predict(x_test)
predictions[1] = lr_model.predict(x_test)
predictions[2] = rf_model.predict(x_test)
final_predicts = []

for i in range(len(predictions[0])):
    count_0 = 0
    count_1 = 0
    for j in range(3):
        if predictions[j][i] == 0: count_0 += 1
        else: count_1 += 1
    final_predicts.append(0 if count_0 > count_1 else 1)

print(classification_report(y_test, final_predicts))
auc_score = roc_auc_score(y_test, final_predicts)
metrics = {"test_auc": auc_score}
print(f"metrics: {metrics}")

              precision    recall  f1-score   support

           0       0.82      0.91      0.87      8333
           1       0.74      0.55      0.63      3667

    accuracy                           0.80     12000
   macro avg       0.78      0.73      0.75     12000
weighted avg       0.80      0.80      0.79     12000

metrics: {'test_auc': 0.7329585738651798}


In [36]:
# mlflow log and end experiment
mlflow.log_params(model.get_params())
mlflow.log_metrics(metrics)
signature = infer_signature(x_test, predictions)
mlflow.sklearn.log_model(
    sk_model=model,
    artifact_path='model',
    signature=signature,
)
mlflow.end_run()

/workspaces/mlops-mara-sample-public/.conda/lib/python3.9/site-packages/mlflow/models/signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)
/home/codespace/.local/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
